# Setup Web Automation with Selenium
Subway Surfers is available on Poki.com. Using Selenium to launch the game and interact with it.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import cv2
import mediapipe as mp
import pyautogui

# Setup browser driver
driver = webdriver.Chrome()
driver.get("https://poki.com/en/g/subway-surfers")
time.sleep(5)  # Allow game page to load

# Ensure the game window is focused
game_window = driver.find_element("tag name", "body")
game_window.click()

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

# Capture Hand Gestures Map Gestures and Integrate Everything

Using OpenCV & MediaPipe to detect fingers


In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import time
import threading
import queue

# --- Configuration ---
WEBCAM_RESOLUTION = (1280, 720) # Use a higher resolution for better accuracy
CONFIDENCE_THRESHOLD = 0.8     # Higher confidence to reduce false positives
ACTION_COOLDOWN = 0.35         # Seconds between actions for smoother control

# --- MediaPipe Hands Initialization ---
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=CONFIDENCE_THRESHOLD,
    min_tracking_confidence=CONFIDENCE_THRESHOLD
)
mp_draw = mp.solutions.drawing_utils

# Frame queue for communication between threads
frame_queue = queue.Queue(maxsize=1)

def video_capture_thread():
    """
    Dedicated thread to capture frames from the webcam as fast as possible.
    """
    print("📹 Starting video capture thread...")
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, WEBCAM_RESOLUTION[0])
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, WEBCAM_RESOLUTION[1])
    cap.set(cv2.CAP_PROP_FPS, 30)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # If the queue is full, remove the old frame and add the new one
        if frame_queue.full():
            frame_queue.get_nowait()
        frame_queue.put(frame)
    cap.release()
    print("Video capture thread finished.")

def perform_action(finger_count):
    """Maps finger count to a keyboard press and prints the action."""
    actions_map = {
        1: 'up',    # Jump
        2: 'down',  # Roll
        3: 'right', # Move Right
        4: 'left',  # Move Left
    }
    action = actions_map.get(finger_count)
    if action:
        print(f"🖐️ Detected {finger_count} fingers. Action: {action.upper()}")
        pyautogui.press(action)

def main_gesture_loop():
    """
    Main loop to process frames, detect gestures, and control the game.
    """
    last_action_time = 0

    # Start the video capture in a separate thread
    capture_thread = threading.Thread(target=video_capture_thread, daemon=True)
    capture_thread.start()
    time.sleep(2) # Give the camera time to initialize

    print("🚀 Gesture detection started. Press 'q' in the OpenCV window to quit.")

    while True:
        if frame_queue.empty():
            continue

        frame = frame_queue.get()
        frame = cv2.flip(frame, 1)
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)

        finger_count = 0
        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0]

            # Draw landmarks for visualization
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # --- Refined Finger Counting Logic ---
            finger_tip_ids = [8, 12, 16, 20] # Tips of index, middle, ring, pinky
            fingers_up = 0

            # Check four fingers (up/down)
            for tip_id in finger_tip_ids:
                if hand_landmarks.landmark[tip_id].y < hand_landmarks.landmark[tip_id - 2].y:
                    fingers_up += 1

            # Check thumb (left/right) - more robust for horizontal hand
            if hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x:
                fingers_up += 1

            finger_count = fingers_up

            # --- Action Cooldown Logic ---
            current_time = time.time()
            if current_time - last_action_time > ACTION_COOLDOWN:
                perform_action(finger_count)
                last_action_time = current_time

            # Display finger count on the frame
            cv2.putText(frame, f"Fingers: {finger_count}", (30, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 3, cv2.LINE_AA)

        # Display the frame
        cv2.imshow("Gesture Controller - Press 'q' to quit", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()
    print("🛑 Gesture detection stopped.")

if __name__ == "__main__":
    main_gesture_loop()


📹 Starting video capture thread...
🚀 Gesture detection started. Press 'q' in the OpenCV window to quit.
🖐️ Detected 3 fingers. Action: RIGHT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 2 fingers. Action: DOWN
🖐️ Detected 2 fingers. Action: DOWN
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 3 fingers. Action: RIGHT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 4 fingers. Action: LEFT
🖐️ Detected 2 fingers. Action: DOWN
🖐️ Detected 2 fingers. Action: